In [ ]:
from torchvision.models import resnet34 
from torchvision.datasets import ImageFolder
from sklearn.model_selection import train_test_split
from torch.utils.data import SubsetRandomSampler, DataLoader
from torch.optim import Adam
from skimage import transform
from torchvision import transforms
from torchvision.utils import make_grid
from torch import nn 
import numpy as np
import torch

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
if torch.cuda.is_available():  
    dev = "cuda:0" 
else:  
    dev = "cpu" 
device = torch.device(dev) 

In [ ]:
composed = transforms.Compose(
    [transforms.ToTensor(), transforms.Resize((224, 224)),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [ ]:
dataset = ImageFolder(root='root', transform=composed)

In [ ]:
targets = dataset.targets

In [ ]:
train_indexes, test_indexes = train_test_split(np.arange(len(targets)), test_size=0.2, shuffle=True, stratify=targets)

In [ ]:
train_sampler = SubsetRandomSampler(train_indexes)
test_sampler = SubsetRandomSampler(test_indexes)

In [ ]:
batch_size = 32

In [ ]:
train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
test_loader = DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)

In [ ]:
net = resnet34(pretrained=True)

In [ ]:
net.fc = nn.Linear(net.fc.in_features, 18)

In [ ]:
net = net.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = Adam(net.parameters(), lr=0.01)

In [ ]:
for epoch in range(3):
    running_loss = 0.0
    j = 0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0], data[1]
        optimizer.zero_grad()
        outputs = net(inputs.to(device))
        loss = criterion(outputs, labels.to(device))
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        j += 1
        if (i + 1) % 100 == 0:
            print(f'Epoch: {epoch + 1}, {i + 1}/{len(train_loader)}, loss: {running_loss / j}')
            running_loss = 0.0
            j = 0

print('Finished')

In [ ]:
correct = 0
total = 0
i = 0
with torch.no_grad():
    for data in test_loader:
        if (i + 1) % 25 == 0:
            print(f'{i + 1}/{len(test_loader)}')
        i += 1
        images, labels = data[0], data[1]
        outputs = net(images.to(device))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.to(device)).sum().item()
print(f'Accuracy: {100 * correct / total} %')